In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs
import gc

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
from tqdm import tqdm

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
# FUNDAMENTAÇÃO (para a escolha do período)
# https://www2.ufjf.br/noticias/2021/04/20/infectologista-explica-riscos-de-reinfeccao-pelo-coronavirus-e-suas-variantes/

In [ ]:
def find_best_record(id_not_1, cns_1, cpf_1, diagnostico_1, evolucao_1, id_not_2, cns_2, cpf_2, diagnostico_2, evolucao_2):
    if ((cns_1 == None) and (cpf_1 == None)):
        if ((cns_2 != None) or (cpf_2 != None)):
            #TROCA AS FICHAS
            id_temp = id_not_1
            diag_temp = diagnostico_1
            evolucao_temp = evolucao_1


            id_not_1 = id_not_2
            diagnostico_1 = diagnostico_2
            evolucao_1 = evolucao_2


            id_not_2 = id_temp
            diagnostico_2 = diag_temp
            evolucao_2 = evolucao_temp
    
    return id_not_1, diagnostico_1, evolucao_1, id_not_2, diagnostico_2, evolucao_2

### 1. Determinação das FICHAS DUPLICADAS no Notifica

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
update = False
load_downloaded = False

# Load 
notifica = Notifica()
notifica.databases()
notifica.load('notifica', compress=False)
notifica.df = notifica.df.drop_duplicates('id', keep='last')
    
if update:
    mb = Metabase()
    days = 2
    intervalo = f"(data_notificacao >= NOW() - INTERVAL '{days} DAY') or (data_liberacao >= NOW() - INTERVAL '{days} DAY') or (updated_at >= NOW() - INTERVAL '{days} DAY') or (data_coleta >= NOW() - INTERVAL '{days} DAY') or (data_encerramento >= NOW() - INTERVAL '{days} DAY') or (data_cura_obito >= NOW() - INTERVAL '{days} DAY')"
    mb.generate_notifica_query('update_notifica', where=intervalo, replace=True)
    update_notifica_parts = mb.download_notificacao('update_notifica', load=load_downloaded)
    
    update_notifica = Notifica()
    update_notifica.read(update_notifica_parts)
    update_notifica.normalize()
    
    
    update_notifica.df = update_notifica.df.drop_duplicates('id', keep='last')
    analise_dtd = update_notifica.analise_data_diagnostico() ## only in casos confirmados, pd.NaT nos demais

#     novas_notificacoes, atualizacoes_fichas = notifica.update(update_notifica)#, observed_cols=['excluir_ficha','status_notificacao','classificacao_final','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito','data_encerramento'])
    notifica.update(update_notifica)

    notifica.fix_dtypes()
    notifica.save(replace=True, compress=False)

notifica.df.shape

In [ ]:
# MUDANÇA DE VALORES PARA CURITIBA, ficha inativa volta à ativa
notifica.df.loc[(notifica.df['ibge_residencia'] == 410690) & (notifica.df['status_notificacao'] == 3), 'status_notificacao'] = 2

In [ ]:
notifica.df = notifica.df.loc[((notifica.df['classificacao_final']==2)&(notifica.df['excluir_ficha']==2)&(notifica.df['status_notificacao'].isin([1,2])))]

notifica.df = pd.merge(notifica.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica.df = pd.merge(notifica.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

# notifica.df['exame_nome'] = notifica.replace('exame',inplace=False)
notifica.replace('sexo')

notifica.df = notifica.df.loc[((notifica.df['sexo']!='N')&(notifica.df['mun_resid'].notna())&(notifica.df['data_diagnostico'].notna())&(notifica.df['paciente'].str.len() > 5))]

display(notifica.df[['id','evolucao']].groupby(['evolucao']).count())

In [ ]:
notifica_duplicados = notifica.check_duplicates(keep=False)
notifica_duplicados.reset_index(inplace=True)
notifica_duplicados.groupby(['ids_len'])[['ids_len']].count()

In [ ]:
duplicados_only = notifica_duplicados.drop_duplicates(subset='ids_str')
duplicados_only = duplicados_only.loc[duplicados_only['duplicated_cols'].str.contains('hash_mae')]
duplicados_only = duplicados_only.loc[duplicados_only['duplicated_ncols'] > 1]
# duplicados_only = duplicados_only.loc[duplicados_only['ids_len'] == 2]
duplicados_only = duplicados_only.sort_values(by='ids_len', ascending=True)
duplicados_only

In [ ]:
del(notifica.df)
gc.collect()

### 2. Início do MÉTODO

In [ ]:
notifica_untouched = Notifica()
notifica_untouched.load('notifica', compress=False)
notifica_untouched.df = notifica_untouched.df.drop_duplicates('id', keep='last')
notifica_untouched.df.shape

In [ ]:
try: 
    notifica_untouched.df = notifica_untouched.df.drop(columns={'reinfeccao', 'reinfeccao_ids'})
except:
    pass
notifica_untouched.df.shape

In [ ]:
for i in tqdm(duplicados_only.index):
    list_verify = duplicados_only.loc[i, 'ids']
    df_temp = notifica_untouched.df.loc[notifica_untouched.df['id'].isin(list_verify)].sort_values('data_diagnostico')

    for j in range(0, len(list_verify) - 1, 1):
        #FICHA 1, provavelmente já está no cc
        id_not_1 = df_temp.loc[df_temp.index[j], 'id']
        cns_1 = df_temp.loc[df_temp.index[j], 'cns']
        cpf_1 = df_temp.loc[df_temp.index[j], 'cpf']
        diagnostico_1 = df_temp.loc[df_temp.index[j], 'data_diagnostico']
        evolucao_1 = df_temp.loc[df_temp.index[j], 'evolucao']
        
        
        #FICHA 2, provavelmente NÃO está no cc
        id_not_2 = df_temp.loc[df_temp.index[j+1], 'id']
        cns_2 = df_temp.loc[df_temp.index[j+1], 'cns']
        cpf_2 = df_temp.loc[df_temp.index[j+1], 'cpf']
        diagnostico_2 = df_temp.loc[df_temp.index[j+1], 'data_diagnostico']
        evolucao_2 = df_temp.loc[df_temp.index[j+1], 'evolucao']


        id_not_1, diagnostico_1, evolucao_1, id_not_2, diagnostico_2, evolucao_2 = find_best_record(id_not_1, cns_1, cpf_1, diagnostico_1, evolucao_1, id_not_2, cns_2, cpf_2, diagnostico_2, evolucao_2)

        
        diff = diagnostico_2 - diagnostico_1
        if ((abs(diff.days) >= 90) and (evolucao_2 != 2)):
            notifica_untouched.df.loc[notifica_untouched.df['id'] == id_not_2, ('reinfeccao', 'reinfeccao_ids')] = (True, id_not_1)

In [ ]:
notifica_untouched.df['reinfeccao'] = notifica_untouched.df['reinfeccao'].fillna(False)

In [ ]:
notifica_untouched.df.loc[notifica_untouched.df['reinfeccao'] == True].shape

In [ ]:
notifica_untouched.df.loc[(notifica_untouched.df['reinfeccao'] == True) & (notifica_untouched.df['status_notificacao']==3)].shape

In [ ]:
# 33102 rows × 190 columns

### 3. Salvar os arquivos necessários

In [ ]:
path_db = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'notifica')

In [ ]:
duplicados_only.to_excel(join(path_db, 'relacao_duplicados_notifica.xlsx'))

In [ ]:
notifica_untouched.save(replace=True, compress=False)

In [ ]:
# TEMPO DE EXECUÇÃO

now = pd.to_datetime(datetime.today())
final = pd.to_datetime(datetime.today())
delta = final - timedelta(minutes=agora.minute, hours=agora.hour)
print(f"início: {agora.strftime('%d/%m/%Y')}\t{agora.strftime('%H:%M')}")
print(f"agora: {now.strftime('%d/%m/%Y')}\t{now.strftime('%H:%M')}")
print(f"tempo corrido: {delta.strftime('%H:%M:%S')}")